In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.svm import SVR
import numpy as np
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.base import TransformerMixin

In [34]:
hourly=pd.read_csv('./hour.csv')
daily=pd.read_csv('./hour.csv')

In [3]:
hourly.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [4]:
daily.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [41]:
#X1_Train = hourly[hourly['dteday'] < '2012-10-01'].drop(['cnt','registered','dteday'],axis=1) ## CASUAL IN DATA
#X2_Train =  hourly[hourly['dteday'] < '2012-10-01'].drop(['cnt','casual','dteday'],axis=1) ## REGISTERED IN DATA 
#X3_Train = hourly[hourly['dteday'] < '2012-10-01'].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THEM IN DATA

#X1_Test = hourly[hourly['dteday'] >= '2012-10-01'].drop(['cnt','registered','dteday'],axis=1)## CASUAL IN DATA
#X2_Test =  hourly[hourly['dteday'] >='2012-10-01'].drop(['cnt','casual','dteday'],axis=1)## REGISTERED IN DATA
#X3_Test = hourly[hourly['dteday'] >= '2012-10-01'].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THEM IN DATA

#y_cas_train =hourly['casual'][hourly['dteday'] < '2012-10-01']
#y_cnt_train =hourly['cnt'][hourly['dteday'] < '2012-10-01']
#y_reg_train =hourly['registered'][hourly['dteday'] < '2012-10-01']

#y_cas_test =hourly['casual'][hourly['dteday'] >= '2012-10-01']
#y_cnt_test =hourly['cnt'][hourly['dteday'] >= '2012-10-01']
#y_reg_test =hourly['registered'][hourly['dteday'] >= '2012-10-01']

In [7]:
#print(y_cnt_test)
#print(y_reg_test + y_pred)
#print(np.ravel(y_cnt_test),np.ravel((y_reg_test + y_pred)))

In [8]:
#SVR_Lin = SVR(kernel='linear',C=1e3)
#SVR_Lin.fit(X2_Train, y_cas_train)
#y_pred=SVR_Lin.predict(X2_Test)
#SVR_Lin.score(np.ravel(y_cnt_test),np.ravel((y_reg_test + y_pred)))

In [9]:
#regr_2 = AdaBoostRegressor(regr.fit(X2_Train, y_cas_train))
#regr_2.fit(X2_Train, y_cas_train)
#y_2 = regr_2.predict(X2_Test)

In [50]:
X_Train = hourly[hourly['dteday'] < '2012-07-01'].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THst_EM IN DATA
X_Test = hourly[(hourly['dteday'] >= '2012-07-01') & (hourly['dteday'] <= '2012-09-31')].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THEM IN DATA
Y_cnt_test =hourly['cnt'][(hourly['dteday'] >= '2012-07-01') & (hourly['dteday'] <= '2012-09-31')]
Y_cnt_train =hourly['cnt'][hourly['dteday'] < '2012-07-01']

In [62]:
X_Train1 = hourly[hourly['dteday'] < '2012-10-01'].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THst_EM IN DATA
X_Test1 = hourly[hourly['dteday'] >= '2012-10-01'].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THEM IN DATA
Y_cnt_test1 =hourly['cnt'][hourly['dteday'] >= '2012-10-01']
Y_cnt_train1 =hourly['cnt'][hourly['dteday'] < '2012-10-01']

In [51]:
regr = LinearRegression(fit_intercept =True)
regr.fit(X_Train, Y_cnt_train) 
y_pred = regr.predict(X_Test) 
regr.score(X_Test, Y_cnt_test) 

0.2789473627513266

In [71]:
regr = LinearRegression(fit_intercept =True)
regr.fit(X_Train1, Y_cnt_train1) 
y_pred1 = regr.predict(X_Test1) 
regr.score(X_Test1, Y_cnt_test1) 

0.2939320614896219

In [52]:
polynomial_features=PolynomialFeatures(degree=2)
x_poly=polynomial_features.fit_transform(X_Train)
X_test_poly=polynomial_features.fit_transform(X_Test)
model=LinearRegression()
model.fit(x_poly, Y_cnt_train)
y_poly_pred = model.predict(X_test_poly)
model.score(X_test_poly, Y_cnt_test)

0.371592309474932

In [64]:
polynomial_features=PolynomialFeatures(degree=2)
x_poly=polynomial_features.fit_transform(X_Train1)
X_test_poly=polynomial_features.fit_transform(X_Test1)
model=LinearRegression()
model.fit(x_poly, Y_cnt_train1)
y_poly_pred1 = model.predict(X_test_poly)
model.score(X_test_poly, Y_cnt_test1)

0.46424469832833104

In [85]:
dataset = pd.DataFrame({'y_poly_pred':y_poly_pred,'Y_pred':y_pred,'actual':Y_cnt_test})
dataset.reset_index(drop=True,inplace=True)
dataset.head()

,y_poly_pred,Y_pred,actual
0,29.407885,230.153527,149
1,69.111436,226.145672,93
2,96.641895,223.514035,90
3,113.286301,218.841103,33
4,148.528556,223.929032,4


In [99]:
dataset1 = pd.DataFrame({'y_poly_pred':y_poly_pred1,'Y_pred':y_pred1,'actual':Y_cnt_test1})
dataset1.reset_index(drop=True,inplace=True)
dataset1.head()
furqan=pd.concat([dataset,dataset1],axis=0)
furqan.reset_index(drop=True,inplace=True)
furqan

,y_poly_pred,Y_pred,actual
0,29.407885,230.153527,149
1,69.111436,226.145672,93
2,96.641895,223.514035,90
3,113.286301,218.841103,33
4,148.528556,223.929032,4
5,156.065910,220.017959,10
6,191.674329,234.717626,27
7,236.289601,263.547506,50
8,316.361891,299.439080,142
9,354.001047,326.270073,219


In [102]:
date =hourly['dteday'][hourly['dteday'] >= '2012-07-01'] 
date=pd.DataFrame(date)
date.reset_index(drop=True,inplace=True)


In [103]:
two.reset_index(drop=True,inplace=True)
new_dataset=pd.concat([date,furqan],axis=1)

In [100]:
X_Train2 = new_dataset[new_dataset['dteday'] < '2012-10-01'].drop(['actual','dteday'],axis=1)## NONE OF THst_EM IN DATA
X_Test2 = new_dataset[new_dataset['dteday'] >= '2012-10-01'].drop(['actual','dteday'],axis=1)## NONE OF THEM IN DATA
Y_cnt_test2 =new_dataset['actual'][new_dataset['dteday'] >= '2012-10-01']
Y_cnt_train2 =new_dataset['actual'][new_dataset['dteday'] < '2012-10-01']

In [101]:
model= GradientBoostingRegressor()
model.fit(X_Train2, Y_cnt_train2)
model.score(X_Test2, Y_cnt_test2)

0.3114018763793841